In [20]:
import tweepy
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
from nltk.stem.porter import PorterStemmer as PS

In [21]:
config = pd.read_csv("./config.csv")

In [22]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [23]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [24]:
# キーワードからツイートを取得
s='btc exclude:retweets'
tweets = tweepy.Cursor(api.search,  
                        count=1,
                        q=s,
                        exclude_replies=True,
                        tweet_mode='extended',
                        show_user=True,
                        lang = 'en',
                        since='2021-07-08',
                        until='2021-07-13',
                        ).items(100);


In [25]:
#ツイートの配列
t_data = []
for tweet in tweets:
    t_data.append([tweet.user.name,'@'+tweet.user.screen_name,tweet.user.followers_count,tweet.full_text.replace('\n',''),tweet.favorite_count,tweet.retweet_count])

In [26]:
df = pd.DataFrame(data=t_data, columns=['Name','User Name','Followers','Tweet','Favorite','RT'])
df.head(10)

,Name,User Name,Followers,Tweet,Favorite,RT
0,Mario,@XCASH_HIPPIE,14,Current stats of DELEGATE_DONT_HATERank: 27Blo...,1,0
1,Freedom 45,@Freedom45ASAP,57,@Silver_Gold_BTC @uselinkinv Abra anywhere aro...,3,0
2,Bitcoin,@BitcoinBlockBot,301,"Block #: 690,763Time: 7/12/2021, 11:57:46 PM U...",0,0
3,BitcoinAgile,@bitcoinagile,59353,BTC - we might need to wait a little bit more ...,0,0
4,The Bitcoin Street Journal,@BTCstJournal,10733,BTC – we might need to wait a little bit more ...,1,0


In [27]:
df.tail(10)

,Name,User Name,Followers,Tweet,Favorite,RT
0,Mario,@XCASH_HIPPIE,14,Current stats of DELEGATE_DONT_HATERank: 27Blo...,1,0
1,Freedom 45,@Freedom45ASAP,57,@Silver_Gold_BTC @uselinkinv Abra anywhere aro...,3,0
2,Bitcoin,@BitcoinBlockBot,301,"Block #: 690,763Time: 7/12/2021, 11:57:46 PM U...",0,0
3,BitcoinAgile,@bitcoinagile,59353,BTC - we might need to wait a little bit more ...,0,0
4,The Bitcoin Street Journal,@BTCstJournal,10733,BTC – we might need to wait a little bit more ...,1,0


In [19]:
#中身の確認
for tweet in tweepy.Cursor(api.search, q='緊急事態宣言 min_faves:400 min_retweets:400 exclude:retweets').items(10):
    print(tweet)


Status(_api=<tweepy.api.API object at 0x0000016C9FD5D520>, _json={'created_at': 'Wed Jul 14 01:40:25 +0000 2021', 'id': 1415123990629158912, 'id_str': '1415123990629158912', 'text': '広島県原水協、バッハ氏の広島訪問に抗議、五輪中止を求める声明を発表。\n緊急事態宣言の東京から多数の人々を引き連れて広島に移動することは納得が得られず、「政治利用」と批判されても仕方ないと。\n当然の批判だ。\n被爆地を五輪強行開催の道具に政治利用するなど、まったくもって許されない。', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1409223888, 'id_str': '1409223888', 'name': '志位和夫', 'screen_name': 'shiikazuo', 'location': '', 'description': '日本共産党委員長の志位和夫です。力をあわせて一緒に政治を変えましょう。', 'url': 'http://t.co/Yp1lqKMNWj', 'entities': {'url': {'urls': [{'url': 'http://t.co/Yp1lqKMNWj', 

In [13]:
#ステミング
from nltk.stem.porter import PorterStemmer as PS
ps = PS()
ps.stem("I'm looking forward to seeing you ")

"i'm looking forward to seeing you "